In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab Notebooks/3yr_pjt

/content/drive/MyDrive/Colab Notebooks/3yr_pjt


Import modeules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os

# Data pre-processing

## Stock price dataset preprocessing

In [ ]:
# Set the path for the original stock data and the directory for the preprocessed data
stock_dataset = './data/stock_data'
preprocessed_dataset = './data/stock_price_data'

# Create a directory for storing preprocessed data if it does not exist
if not os.path.exists(preprocessed_dataset):
    os.makedirs(preprocessed_dataset)

# Loop through each file in the original stock data directory
for file in os.listdir(stock_dataset):
    file_path = os.path.join(stock_dataset, file)
    if file_path.endswith('.csv'):
        # Read the CSV file, setting 'Date' as the index column and parsing dates
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Replace 0 values in 'Volume' and 'Close' columns with NaN and then drop rows with any NaN values
        df['Volume'] = df['Volume'].replace(0, np.nan)
        df['Close'] = df['Close'].replace(0, np.nan)
        df = df.dropna()

        # Define the path for saving the preprocessed file
        preprocessed_file_path = os.path.join(preprocessed_dataset, file)

        # Save the preprocessed data to a new CSV file
        df.to_csv(preprocessed_file_path)

## Feature dataset preprocessing (for Deep Learning)

Preprocess individual market index datasets (S&P 500, Dow Jones Industrial Average (DJI), Nasdaq (IXIC)) in a similar manner

Replace 0 values, drop NaNs, remove commas, and convert to float

### S&P 500 dataset preprocessing

In [ ]:
snp500_dataset = 'data/snp500.csv'
df = pd.read_csv(snp500_dataset, index_col='Date', parse_dates=True)

df = df['Close*'].replace(0, np.nan)
df = df.dropna()
df = df.str.replace(',', '').astype(float)

df.to_csv('data/snp500_preprocessed.csv')

### Dow Jones Industrial Average (DJI) dataset preprocessing

In [ ]:
dji_dataset = 'data/DJI.csv'
df = pd.read_csv(dji_dataset, index_col='Date', parse_dates=True)

df = df['Close*'].replace(0, np.nan)
df = df.dropna()
df = df.str.replace(',', '').astype(float)

df.to_csv('data/dji_preprocessed.csv')

### Nasdaq (IXIC) dataset preprocessing

In [ ]:
nasdaq_dataset = 'data/IXIC.csv'
df = pd.read_csv(snp500_dataset, index_col='Date', parse_dates=True)

df = df['Close*'].replace(0, np.nan)
df = df.dropna()
df = df.str.replace(',', '').astype(float)

df.to_csv('data/ixic_preprocessed.csv')

## Combine feature datasets with stock dataset

In [ ]:
# Load the preprocessed market index datasets
snp500_preprocessed = pd.read_csv('data/snp500_preprocessed.csv', index_col='Date', parse_dates=True)
dji_preprocessed = pd.read_csv('data/dji_preprocessed.csv', index_col='Date', parse_dates=True)
nasdaq_preprocessed = pd.read_csv('data/ixic_preprocessed.csv', index_col='Date', parse_dates=True)

# Find the latest start date among the three datasets to align them temporally
latest_start_date = max(snp500_preprocessed.index.min(), dji_preprocessed.index.min(), nasdaq_preprocessed.index.min())

# Trim each dataset to start from this latest start date for consistency
snp500_preprocessed = snp500_preprocessed[snp500_preprocessed.index >= latest_start_date]
dji_preprocessed = dji_preprocessed[dji_preprocessed.index >= latest_start_date]
nasdaq_preprocessed = nasdaq_preprocessed[nasdaq_preprocessed.index >= latest_start_date]

# Rename columns to clearly indicate the source of the data
snp500_preprocessed.rename(columns={'Close*': 'Close_snp500'}, inplace=True)
dji_preprocessed.rename(columns={'Close*': 'Close_dji'}, inplace=True)
nasdaq_preprocessed.rename(columns={'Close*': 'Close_nasdaq'}, inplace=True)

preprocessed_dataset = './data/stock_price_data'

# Create a directory for the final merged datasets if it does not exist
deep_dataset = './data/deep_stock_data'
if not os.path.exists(deep_dataset):
    os.makedirs(deep_dataset)

# Merge each individual stock file with the preprocessed market index data
for file in os.listdir(preprocessed_dataset):
    file_path = os.path.join(preprocessed_dataset, file)
    if file.endswith('.csv'):
        stock_df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
        stock_df = stock_df[stock_df.index >= latest_start_date]

        # Join the stock data with each of the market index datasets
        merged_df = stock_df.join(snp500_preprocessed, how='left', rsuffix='_snp500')
        merged_df = merged_df.join(dji_preprocessed, how='left', rsuffix='_dji')
        merged_df = merged_df.join(nasdaq_preprocessed, how='left', rsuffix='_nasdaq')

        # Forward-fill any missing values in the market index columns
        for col in ['Close_snp500', 'Close_dji', 'Close_nasdaq']:
            merged_df[col] = merged_df[col].fillna(method='ffill')

        # Save the merged dataset to a new CSV file in the final directory
        merged_file_path = os.path.join(deep_dataset, file)
        merged_df.to_csv(merged_file_path)

### Normalization for deep learning

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Set the directory containing the deep preprocessed stock data
deep_dataset = './data/deep_stock_data'
# Initialize an empty DataFrame to combine all stock data
combined_df = pd.DataFrame()

# Combine data from all CSV files in the deep dataset directory into a single DataFrame
for file in os.listdir(deep_dataset):
    file_path = os.path.join(deep_dataset, file)
    # Ensure only CSV files are processed
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
        combined_df = pd.concat([combined_df, df])

# Select specific columns for scaling
combined_df = combined_df[['Volume', 'Close', 'Close_snp500', 'Close_dji', 'Close_nasdaq']]


### Save Scaler

In [ ]:
# Dictionary to store scalers for each column
scalers = {}
for col in combined_df.columns:
    # Initialize a MinMaxScaler for each column
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Fit the scaler to the column data
    scalers[col] = scaler.fit(combined_df[[col]])

    # Save each scaler to a file for later use
    joblib.dump(scaler, f'./scalers/{col}_scaler.pkl')


In [ ]:
# Directory for storing normalized datasets
normalized_dataset = './data/normalized_stock_data'
# Create the directory if it does not exist
if not os.path.exists(normalized_dataset):
    os.makedirs(normalized_dataset)

# Normalize and save each individual stock file using the fitted scalers
for file in os.listdir(deep_dataset):
    file_path = os.path.join(deep_dataset, file)
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Apply the saved scalers to the respective columns
        for col in scalers:
            scaler = scalers[col]
            df[col] = scaler.transform(df[[col]])

        # Define the path for the normalized file
        normalized_file_path = os.path.join(normalized_dataset, file)
        # Save the normalized data to a new CSV file
        df.to_csv(normalized_file_path)

# Train & Evaluate Model

## Accuracy index

In [ ]:
# Function to calculate Mean Squared Error (MSE)
def get_mse(actual, forecast):
    actual = np.where(actual < 0, 0, actual)
    forecast = np.where(forecast < 0, 0, forecast)
    actual = actual + 0.0001
    forecast = forecast + 0.0001
    squared_diff = np.square(actual - forecast)
    return np.mean(squared_diff)

# Function to calculate Mean Absolute Percentage Error (MAPE)
def get_mape(actual, forecast):
    actual = np.where(actual<0, 0, actual)
    forecast = np.where(forecast<0, 0, forecast)
    actual = actual + 0.0001
    forecast = forecast + 0.0001
    return np.mean(np.abs((actual - forecast) / actual)) * 100

# Function to calculate Directional Accuarcy
def get_directional_accuracy(actual, forecast):
    # Calculate direction of actual price changes
    actual_direction = np.sign(np.diff(actual))
    # Calculate direction of forecasted price changes
    forecast_direction = np.sign(np.diff(forecast))

    correct_predictions = (actual_direction == forecast_direction).sum()
    total_predictions = len(actual_direction)
    directional_accuracy = correct_predictions / total_predictions
    return directional_accuracy

## Prophet

In [ ]:
from prophet import Prophet

In [ ]:
def evaluate_prophet():
    stock_price = './data/stock_price_data/'
    accuracy_df = pd.DataFrame(columns=['TICKER', 'mse', 'mape'])

    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)

        # Extract ticker from the file name
        ticker = os.path.splitext(file)[0]

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Extract 'Close' column and reset the index
        df_prophet = df['Close'].reset_index()

        # Rename the columns to match Prophet's requirements
        df_prophet.columns = ['ds', 'y']

        # Convert the 'ds' column to datetime
        df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

        # Split the data into train and evaluate sets
        cutoff_date = '2023-01-03'
        train = df_prophet[df_prophet['ds'] < cutoff_date]
        evaluate = df_prophet[df_prophet['ds'] >= cutoff_date]

        # Create and fit the Prophet model using the training data
        model = Prophet()
        model.fit(train)

        # Make future DataFrame for forecasting
        future = model.make_future_dataframe(periods=len(evaluate))

        # Forecast using the fitted model
        forecast = model.predict(future)

        # Extract actual and forecast values for evaluation
        y_true = evaluate['y'].values
        fcst = forecast.iloc[-len(evaluate):,:][['ds', 'yhat']]

        # Save the prediction data to a CSV file
        prediction_file_name = f"./prediction/{ticker}_prophet.csv"
        fcst.to_csv(prediction_file_name, index=False)

        # Calculate accuracy
        y_pred = fcst['yhat'].values
        mse = get_mse(y_true, y_pred)
        mape = get_mape(y_true, y_pred)
        directional_accuracy = get_directional_accuracy(y_true, y_pred)

        # Store accuracy values in the DataFrame
        accuracy_df = pd.concat([accuracy_df, pd.DataFrame([[ticker, mse, mape, directional_accuracy]],
                                                    columns=['TICKER', 'mse', 'mape', 'directional_accuracy'])], ignore_index=True)

    return(accuracy_df)

In [ ]:
accuracy_prophet = evaluate_prophet()
accuracy_prophet.to_csv("prophet.csv")

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpxv1i06rh/hc9li_lg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpxv1i06rh/1c1_wpeg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=34650', 'data', 'file=/tmp/tmpxv1i06rh/hc9li_lg.json', 'init=/tmp/tmpxv1i06rh/1c1_wpeg.json', 'output', 'file=/tmp/tmpxv1i06rh/prophet_modelarctn22w/prophet_model-20240311181119.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:11:19 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:11:32 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpxv

## ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima

In [ ]:
def evaluate_arima():
    stock_price = './data/stock_price_data/'
    accuracy_df = pd.DataFrame(columns=['TICKER', 'mse', 'mape'])

    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)

        # Extract ticker from the file name
        ticker = os.path.splitext(file)[0]

        # Read the CSV file into a DataFrame
        df_arima = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Extract 'Close' column and reset the index
        df_arima = df_arima['Close'].reset_index()

        # Rename the columns for ARIMA
        df_arima.columns = ['ds', 'y']

        # Convert the 'ds' column to datetime
        df_arima['ds'] = pd.to_datetime(df_arima['ds'])

        # Split the data into train and evaluate sets
        cutoff_date = '2023-01-01'
        train = df_arima[df_arima['ds'] < cutoff_date]
        evaluate = df_arima[df_arima['ds'] >= cutoff_date]

        # Fit ARIMA model using the training data with auto_arima
        model_auto_arima = auto_arima(train['y'], start_p=1, start_q=1,
                                      test='adf',
                                      max_p=3, max_q=3,
                                      m=1,
                                      d=None,
                                      seasonal=False,
                                      start_P=0,
                                      D=0,
                                      trace=True,
                                      error_action='ignore',
                                      suppress_warnings=True,
                                      stepwise=True)

        # Forecast using the fitted ARIMA model
        y_pred = model_auto_arima.predict(n_periods=len(evaluate))

        # Save the prediction data to a CSV file
        prediction_file_name = f"./prediction/{ticker}_arima.csv"
        pd.DataFrame({'ds': evaluate['ds'], 'yhat': y_pred}).to_csv(prediction_file_name, index=False)

        # Calculate accuracy
        mse = get_mse(evaluate['y'].values, y_pred)
        mape = get_mape(evaluate['y'].values, y_pred)
        directional_accuracy = get_directional_accuracy(evaluate['y'].values, y_pred)

        # Update DataFrame with new metrics
        accuracy_df = pd.concat([accuracy_df, pd.DataFrame([[ticker, mse, mape, directional_accuracy]],
                                                    columns=['TICKER', 'mse', 'mape', 'directional_accuracy'])], ignore_index=True)

    return(accuracy_df)

In [ ]:
accuracy_arima = evaluate_arima()
accuracy_arima.to_csv("arima.csv")

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=41010.718, Time=10.74 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=41067.835, Time=1.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=41015.075, Time=3.58 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=41018.450, Time=7.16 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=41069.094, Time=0.82 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=41009.048, Time=12.89 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=41007.116, Time=2.66 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=41008.867, Time=4.09 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=40931.993, Time=21.74 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=41001.722, Time=45.28 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=40799.450, Time=28.48 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=41008.127, Time=17.93 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=41003.421, Time=45.18 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=41006.723, Time=24.34 sec
 ARIMA(3,1,

## LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
# Function to create a sequence dataset
def make_sequence_dataset(feature, label, window_size):
    feature_list = []
    label_list = []

    for i in range(len(feature)-window_size):

        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])

    return np.array(feature_list), np.array(label_list)

In [ ]:
# Function to build an LSTM model with specified input shape
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, activation='tanh', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
def evaluate_lstm():
    # Set the path for the normalized stock data and model saving directory
    stock_price = './data/normalized_stock_data/'
    model_save_path = './models/'
    os.makedirs(model_save_path, exist_ok=True)

    # DataFrame to store accuracy metrics for each stock
    accuracy_df = pd.DataFrame(columns=['TICKER', 'mse', 'mape'])

    # Load the scaler used to normalize the 'Close' prices
    scaler_close = joblib.load('./scalers/Close_scaler.pkl')

    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)
        df = pd.read_csv(file_path, thousands=',')

        # Selecting feature and label columns
        feature_cols = ['Close', 'Volume', 'Close_snp500', 'Close_dji', 'Close_nasdaq']
        label_cols = ['Close']

        # Preparing the feature and label matrices
        feature_df = pd.DataFrame(df, columns=feature_cols)
        label_df = pd.DataFrame(df, columns=label_cols)

        # Converting DataFrame to numpy array for processing
        feature_np = feature_df.to_numpy()
        label_np = label_df.to_numpy()

        # Creating sequences from the data
        window_size = 40
        X, Y = make_sequence_dataset(feature_np, label_np, window_size)

        # Splitting the dataset into training and testing sets
        split = -218
        x_train = X[0:split]
        y_train = Y[0:split]
        x_test = X[split:]
        y_test = Y[split:]

        # Building and training the LSTM model
        model = build_lstm_model(x_train[0].shape)

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

        # Training model with early stopping and learning rate reduction on plateau
        model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=32, callbacks=[early_stopping, reduce_lr])

        # Making predictions
        pred = model.predict(x_test)

        # De-normalizing the predictions and actual values for evaluation
        actual_denormalized = scaler_close.inverse_transform(y_test).flatten()
        pred_denormalized = scaler_close.inverse_transform(pred).flatten()

        # Extracting the ticker symbol from the file name for reference
        ticker = os.path.splitext(file)[0]

        # Saving the trained model
        model_save_filename = f"{model_save_path}{ticker}_lstm_model.h5"
        model.save(model_save_filename)

        # Calculate accuracy metrics (MSE, MAPE, and directional accuracy)
        mse = get_mse(actual_denormalized, pred_denormalized)
        mape = get_mape(actual_denormalized, pred_denormalized)
        directional_accuracy = get_directional_accuracy(actual_denormalized, pred_denormalized)

        # Save the forecast data to a CSV file
        forecast_file_name = f"./prediction/{ticker}_lstm.csv"
        pd.DataFrame({'ds': df['Date'].iloc[-len(pred_denormalized):], 'yhat': pred_denormalized}).to_csv(forecast_file_name, index=False)

        # Store accuracy values in the DataFrame
        accuracy_df = pd.concat([accuracy_df, pd.DataFrame([[ticker, mse, mape, directional_accuracy]], columns=['TICKER', 'mse', 'mape', 'directional_accuracy'])], ignore_index=True)

    return accuracy_df

In [ ]:
accuracy_lstm = evaluate_lstm()
accuracy_lstm.to_csv("lstm.csv")

Epoch 1/30
29/29 [==============================] - 8s 42ms/step - loss: 0.0069 - mae: 0.0591 - val_loss: 5.9588e-04 - val_mae: 0.0243 - lr: 0.0010
Epoch 2/30
29/29 [==============================] - 0s 11ms/step - loss: 4.0613e-04 - mae: 0.0155 - val_loss: 1.8668e-05 - val_mae: 0.0041 - lr: 0.0010
Epoch 3/30
29/29 [==============================] - 0s 9ms/step - loss: 2.2517e-04 - mae: 0.0119 - val_loss: 6.4564e-06 - val_mae: 0.0022 - lr: 0.0010
Epoch 4/30
29/29 [==============================] - 0s 9ms/step - loss: 1.5354e-04 - mae: 0.0098 - val_loss: 1.4091e-06 - val_mae: 0.0010 - lr: 0.0010
Epoch 5/30
29/29 [==============================] - 0s 8ms/step - loss: 1.3484e-04 - mae: 0.0090 - val_loss: 2.9359e-06 - val_mae: 0.0015 - lr: 0.0010
Epoch 6/30
29/29 [==============================] - 0s 9ms/step - loss: 1.2211e-04 - mae: 0.0087 - val_loss: 1.0889e-06 - val_mae: 8.5802e-04 - lr: 0.0010
Epoch 7/30
29/29 [==============================] - 0s 9ms/step - loss: 1.0093e-04 - mae: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
243/243 [==============================] - 7s 12ms/step - loss: 1.7284e-04 - mae: 0.0078 - val_loss: 1.1074e-04 - val_mae: 0.0099 - lr: 0.0010
Epoch 2/30
243/243 [==============================] - 2s 7ms/step - loss: 3.9266e-05 - mae: 0.0044 - val_loss: 1.5822e-04 - val_mae: 0.0123 - lr: 0.0010
Epoch 3/30
243/243 [==============================] - 2s 7ms/step - loss: 2.5230e-05 - mae: 0.0034 - val_loss: 3.5858e-05 - val_mae: 0.0055 - lr: 0.0010
Epoch 4/30
243/243 [==============================] - 2s 7ms/step - loss: 1.9860e-05 - mae: 0.0030 - val_loss: 1.2200e-05 - val_mae: 0.0026 - lr: 0.0010
Epoch 5/30
243/243 [==============================] - 2s 7ms/step - loss: 1.2947e-05 - mae: 0.0024 - val_loss: 1.1423e-05 - val_mae: 0.0026 - lr: 0.0010
Epoch 6/30
243/243 [==============================] - 2s 7ms/step - loss: 1.0621e-05 - mae: 0.0022 - val_loss: 1.4073e-05 - val_mae: 0.0033 - lr: 0.0010
Epoch 7/30
243/243 [==============================] - 2s 9ms/step - loss: 1.1027e

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
208/208 [==============================] - 6s 11ms/step - loss: 2.3140e-04 - mae: 0.0076 - val_loss: 6.9107e-06 - val_mae: 0.0023 - lr: 0.0010
Epoch 2/30
208/208 [==============================] - 2s 9ms/step - loss: 2.2219e-05 - mae: 0.0034 - val_loss: 2.5193e-05 - val_mae: 0.0049 - lr: 0.0010
Epoch 3/30
208/208 [==============================] - 2s 10ms/step - loss: 1.5746e-05 - mae: 0.0029 - val_loss: 5.9116e-06 - val_mae: 0.0021 - lr: 0.0010
Epoch 4/30
208/208 [==============================] - 1s 7ms/step - loss: 1.1968e-05 - mae: 0.0026 - val_loss: 2.9421e-05 - val_mae: 0.0053 - lr: 0.0010
Epoch 5/30
208/208 [==============================] - 2s 7ms/step - loss: 9.3586e-06 - mae: 0.0023 - val_loss: 2.6134e-06 - val_mae: 0.0013 - lr: 0.0010
Epoch 6/30
208/208 [==============================] - 2s 7ms/step - loss: 7.9987e-06 - mae: 0.0021 - val_loss: 1.1984e-06 - val_mae: 9.1905e-04 - lr: 0.0010
Epoch 7/30
208/208 [==============================] - 1s 7ms/step - loss: 6.

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
243/243 [==============================] - 6s 13ms/step - loss: 2.3509e-04 - mae: 0.0071 - val_loss: 5.5188e-06 - val_mae: 0.0019 - lr: 0.0010
Epoch 2/30
243/243 [==============================] - 2s 9ms/step - loss: 2.0682e-05 - mae: 0.0031 - val_loss: 3.8406e-06 - val_mae: 0.0017 - lr: 0.0010
Epoch 3/30
243/243 [==============================] - 2s 8ms/step - loss: 1.3178e-05 - mae: 0.0026 - val_loss: 6.9078e-06 - val_mae: 0.0021 - lr: 0.0010
Epoch 4/30
243/243 [==============================] - 2s 7ms/step - loss: 9.7616e-06 - mae: 0.0022 - val_loss: 1.7621e-05 - val_mae: 0.0038 - lr: 0.0010
Epoch 5/30
243/243 [==============================] - 2s 8ms/step - loss: 7.8774e-06 - mae: 0.0020 - val_loss: 3.2424e-06 - val_mae: 0.0015 - lr: 0.0010
Epoch 6/30
243/243 [==============================] - 2s 7ms/step - loss: 6.6446e-06 - mae: 0.0019 - val_loss: 1.9619e-05 - val_mae: 0.0042 - lr: 0.0010
Epoch 7/30
243/243 [==============================] - 2s 8ms/step - loss: 5.8597e

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
192/192 [==============================] - 5s 12ms/step - loss: 2.4548e-04 - mae: 0.0078 - val_loss: 1.5046e-06 - val_mae: 0.0011 - lr: 0.0010
Epoch 2/30
192/192 [==============================] - 2s 10ms/step - loss: 2.1366e-05 - mae: 0.0033 - val_loss: 2.1970e-06 - val_mae: 0.0013 - lr: 0.0010
Epoch 3/30
192/192 [==============================] - 2s 9ms/step - loss: 1.5004e-05 - mae: 0.0028 - val_loss: 7.9849e-06 - val_mae: 0.0028 - lr: 0.0010
Epoch 4/30
192/192 [==============================] - 1s 7ms/step - loss: 1.1554e-05 - mae: 0.0025 - val_loss: 5.4389e-07 - val_mae: 5.8651e-04 - lr: 0.0010
Epoch 5/30
192/192 [==============================] - 1s 7ms/step - loss: 9.4853e-06 - mae: 0.0023 - val_loss: 2.4315e-06 - val_mae: 0.0014 - lr: 0.0010
Epoch 6/30
192/192 [==============================] - 1s 7ms/step - loss: 8.2184e-06 - mae: 0.0021 - val_loss: 7.9913e-07 - val_mae: 7.3912e-04 - lr: 0.0010
Epoch 7/30
192/192 [==============================] - 1s 7ms/step - loss

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## GRU

In [ ]:
# Function to build a GRU model with specified input shape
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(128, activation='tanh', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
def evaluate_gru():
    # Define paths for data and model saving
    stock_price = './data/normalized_stock_data/'
    model_save_path = './models/'
    os.makedirs(model_save_path, exist_ok=True)

    # DataFrame to store accuracy metrics
    accuracy_df = pd.DataFrame(columns=['TICKER', 'mse', 'mape'])

    # Load the scaler for the 'Close' column to de-normalize predictions
    scaler_close = joblib.load('./scalers/Close_scaler.pkl')

    # Process each file in the normalized stock data directory
    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)
        df = pd.read_csv(file_path, thousands=',')

        # Define feature and label columns
        feature_cols = ['Close', 'Volume', 'Close_snp500', 'Close_dji', 'Close_nasdaq']
        label_cols = ['Close']

        # Prepare feature and label dataframes and convert to numpy arrays
        feature_df = pd.DataFrame(df, columns=feature_cols)
        label_df = pd.DataFrame(df, columns=label_cols)
        feature_np = feature_df.to_numpy()
        label_np = label_df.to_numpy()

        # Create sequence datasets for the GRU model
        window_size = 30
        X, Y = make_sequence_dataset(feature_np, label_np, window_size)

        # Split data into training and testing sets
        split = -218
        x_train = X[0:split]
        y_train = Y[0:split]
        x_test = X[split:]
        y_test = Y[split:]

        # Build and train the GRU model
        model = build_gru_model(x_train[0].shape)

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

        model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=64, callbacks=[early_stopping, reduce_lr])

        # Make predictions with the trained model
        pred = model.predict(x_test)

        # De-normalize the actual and predicted values for evaluation
        actual_denormalized = scaler_close.inverse_transform(y_test).flatten()
        pred_denormalized = scaler_close.inverse_transform(pred).flatten()

        # Extract the ticker symbol from the file name for identification
        ticker = os.path.splitext(file)[0]

        # Save the trained model
        model_save_filename = f"{model_save_path}{ticker}_gru_model.h5"
        model.save(model_save_filename)

        # Evaluate the model
        mse = get_mse(actual_denormalized, pred_denormalized)
        mape = get_mape(actual_denormalized, pred_denormalized)
        directional_accuracy = get_directional_accuracy(actual_denormalized, pred_denormalized)

        # Save prediction results to a CSV file
        forecast_file_name = f"./prediction/{ticker}_gru.csv"
        pd.DataFrame({'ds': df['Date'].iloc[-len(pred_denormalized):], 'yhat': pred_denormalized}).to_csv(forecast_file_name, index=False)

        # Update the accuracy DataFrame with the results
        accuracy_df = pd.concat([accuracy_df, pd.DataFrame([[ticker, mse, mape, directional_accuracy]], columns=['TICKER', 'mse', 'mape', 'directional_accuracy'])], ignore_index=True)

    return accuracy_df

In [ ]:
accuracy_gru = evaluate_gru()
accuracy_gru.to_csv("gru.csv")

Epoch 1/30
15/15 [==============================] - 8s 60ms/step - loss: 0.0138 - mae: 0.0937 - val_loss: 1.3476e-04 - val_mae: 0.0116 - lr: 0.0010
Epoch 2/30
15/15 [==============================] - 0s 12ms/step - loss: 0.0034 - mae: 0.0465 - val_loss: 3.0481e-04 - val_mae: 0.0174 - lr: 0.0010
Epoch 3/30
15/15 [==============================] - 0s 11ms/step - loss: 0.0016 - mae: 0.0315 - val_loss: 7.6139e-05 - val_mae: 0.0087 - lr: 0.0010
Epoch 4/30
15/15 [==============================] - 0s 11ms/step - loss: 0.0010 - mae: 0.0254 - val_loss: 1.4911e-06 - val_mae: 0.0010 - lr: 0.0010
Epoch 5/30
15/15 [==============================] - 0s 11ms/step - loss: 8.5822e-04 - mae: 0.0231 - val_loss: 5.8940e-05 - val_mae: 0.0076 - lr: 0.0010
Epoch 6/30
15/15 [==============================] - 0s 10ms/step - loss: 7.7045e-04 - mae: 0.0218 - val_loss: 1.9804e-06 - val_mae: 0.0012 - lr: 0.0010
Epoch 7/30
15/15 [==============================] - 0s 10ms/step - loss: 5.6683e-04 - mae: 0.0189 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
122/122 [==============================] - 7s 13ms/step - loss: 8.8428e-04 - mae: 0.0171 - val_loss: 2.6116e-05 - val_mae: 0.0039 - lr: 0.0010
Epoch 2/30
122/122 [==============================] - 1s 7ms/step - loss: 1.5054e-04 - mae: 0.0083 - val_loss: 1.1295e-05 - val_mae: 0.0024 - lr: 0.0010
Epoch 3/30
122/122 [==============================] - 1s 7ms/step - loss: 9.0158e-05 - mae: 0.0064 - val_loss: 8.8624e-05 - val_mae: 0.0089 - lr: 0.0010
Epoch 4/30
122/122 [==============================] - 1s 7ms/step - loss: 5.9351e-05 - mae: 0.0051 - val_loss: 2.8863e-05 - val_mae: 0.0048 - lr: 0.0010
Epoch 5/30
122/122 [==============================] - 1s 8ms/step - loss: 4.7113e-05 - mae: 0.0045 - val_loss: 2.0991e-05 - val_mae: 0.0040 - lr: 0.0010
Epoch 6/30
122/122 [==============================] - 1s 7ms/step - loss: 4.1374e-05 - mae: 0.0044 - val_loss: 2.6402e-05 - val_mae: 0.0046 - lr: 0.0010
Epoch 7/30
7/7 [==============================] - 1s 4ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
104/104 [==============================] - 6s 14ms/step - loss: 8.4266e-04 - mae: 0.0167 - val_loss: 2.6451e-04 - val_mae: 0.0161 - lr: 0.0010
Epoch 2/30
104/104 [==============================] - 1s 7ms/step - loss: 1.1335e-04 - mae: 0.0072 - val_loss: 8.2517e-06 - val_mae: 0.0025 - lr: 0.0010
Epoch 3/30
104/104 [==============================] - 1s 8ms/step - loss: 6.3104e-05 - mae: 0.0055 - val_loss: 6.2196e-06 - val_mae: 0.0021 - lr: 0.0010
Epoch 4/30
104/104 [==============================] - 1s 10ms/step - loss: 4.7320e-05 - mae: 0.0048 - val_loss: 7.2916e-06 - val_mae: 0.0024 - lr: 0.0010
Epoch 5/30
104/104 [==============================] - 1s 9ms/step - loss: 3.2652e-05 - mae: 0.0039 - val_loss: 6.1998e-06 - val_mae: 0.0021 - lr: 0.0010
Epoch 6/30
104/104 [==============================] - 1s 9ms/step - loss: 2.6260e-05 - mae: 0.0035 - val_loss: 2.2346e-06 - val_mae: 0.0012 - lr: 0.0010
Epoch 7/30
104/104 [==============================] - 1s 11ms/step - loss: 2.247

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
122/122 [==============================] - 5s 16ms/step - loss: 5.6422e-04 - mae: 0.0137 - val_loss: 5.4275e-05 - val_mae: 0.0069 - lr: 0.0010
Epoch 2/30
122/122 [==============================] - 1s 10ms/step - loss: 7.1190e-05 - mae: 0.0056 - val_loss: 4.1759e-05 - val_mae: 0.0059 - lr: 0.0010
Epoch 3/30
122/122 [==============================] - 1s 11ms/step - loss: 3.7746e-05 - mae: 0.0041 - val_loss: 6.5374e-06 - val_mae: 0.0021 - lr: 0.0010
Epoch 4/30
122/122 [==============================] - 1s 7ms/step - loss: 2.5169e-05 - mae: 0.0034 - val_loss: 5.9020e-06 - val_mae: 0.0020 - lr: 0.0010
Epoch 5/30
122/122 [==============================] - 1s 7ms/step - loss: 1.9392e-05 - mae: 0.0030 - val_loss: 1.4896e-05 - val_mae: 0.0032 - lr: 0.0010
Epoch 6/30
122/122 [==============================] - 1s 8ms/step - loss: 1.6034e-05 - mae: 0.0028 - val_loss: 4.3258e-05 - val_mae: 0.0062 - lr: 0.0010
Epoch 7/30
122/122 [==============================] - 1s 8ms/step - loss: 1.418

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
96/96 [==============================] - 6s 15ms/step - loss: 6.7924e-04 - mae: 0.0153 - val_loss: 7.4705e-06 - val_mae: 0.0026 - lr: 0.0010
Epoch 2/30
96/96 [==============================] - 1s 8ms/step - loss: 9.2009e-05 - mae: 0.0065 - val_loss: 8.0846e-06 - val_mae: 0.0027 - lr: 0.0010
Epoch 3/30
96/96 [==============================] - 1s 7ms/step - loss: 6.3057e-05 - mae: 0.0055 - val_loss: 2.2763e-05 - val_mae: 0.0047 - lr: 0.0010
Epoch 4/30
96/96 [==============================] - 1s 8ms/step - loss: 4.2899e-05 - mae: 0.0044 - val_loss: 5.2037e-06 - val_mae: 0.0022 - lr: 0.0010
Epoch 5/30
96/96 [==============================] - 1s 8ms/step - loss: 3.0775e-05 - mae: 0.0038 - val_loss: 2.1999e-05 - val_mae: 0.0046 - lr: 0.0010
Epoch 6/30
96/96 [==============================] - 1s 8ms/step - loss: 2.6212e-05 - mae: 0.0035 - val_loss: 5.9856e-06 - val_mae: 0.0023 - lr: 0.0010
Epoch 7/30
96/96 [==============================] - 1s 8ms/step - loss: 1.9887e-05 - mae: 0.0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Combine model

## Make dataframe for each ticker's with models' accuracy index

In [ ]:
# Load results from CSV files for each forecasting method
arima_df = pd.read_csv('arima.csv')
prophet_df = pd.read_csv('prophet.csv')
lstm_df = pd.read_csv('lstm.csv')
gru_df = pd.read_csv('gru.csv')

# Assign a 'Method' column to each DataFrame to identify the forecasting method
arima_df['Method'] = 'ARIMA'
lstm_df['Method'] = 'LSTM'
prophet_df['Method'] = 'Prophet'
gru_df['Method'] = 'GRU'

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat([arima_df, lstm_df, prophet_df, gru_df])

# Sort combined_df by MAPE and group by 'TICKER' to find the best method for each ticker
best_methods = combined_df.sort_values('mape').groupby('TICKER').first().reset_index()
# Drop any unnamed column that might be present due to the CSV structure
best_methods.drop('Unnamed: 0', axis=1, inplace=True)

print(best_methods)

    TICKER         mse      mape  directional_accuracy Method
0     AAPL   18.178535  2.099670              0.506912    GRU
1     ABBV  133.216557  5.844732              0.451613   LSTM
2     ABNB  179.449717  8.569236              0.548387    GRU
3      ABT   51.355242  5.778981              0.493088   LSTM
4      ACN   71.520763  2.432209              0.534562    GRU
..     ...         ...       ...                   ...    ...
103   WELL    2.775230  1.705658              0.502304    GRU
104    WFC    2.459892  3.004492              0.502304   LSTM
105    WMT    7.408267  1.425628              0.520737   LSTM
106    XEL    5.958393  3.042083              0.502304    GRU
107    XOM    7.526336  2.000006              0.488479    GRU

[108 rows x 5 columns]


In [ ]:
# Count how many times each method was best across all tickers
method_counts = best_methods['Method'].value_counts()
print(method_counts)

GRU        60
LSTM       43
ARIMA       4
Prophet     1
Name: Method, dtype: int64


In [ ]:
# Calculate and print the total average MAPE across all methods and tickers
average_mape = combined_df['mape'].mean()
print("Total average MAPE:", average_mape)

Total average MAPE: 10.229946022933023


In [ ]:
# Drop any unnamed column and rename columns to clearly indicate the method and metric
arima_df = pd.read_csv('arima.csv')
prophet_df = pd.read_csv('prophet.csv')
lstm_df = pd.read_csv('lstm.csv')
gru_df = pd.read_csv('gru.csv')

# Repeat the loading step to ensure clean data, then adjust and rename
arima_df = arima_df.drop('Unnamed: 0', axis=1)
arima_df.columns = ['TICKER', 'mse_arima', 'mape_arima', 'dir_arima']

gru_df = gru_df.drop('Unnamed: 0', axis=1)
gru_df.columns = ['TICKER', 'mse_gru', 'mape_gru', 'dir_gru']

lstm_df = lstm_df.drop('Unnamed: 0', axis=1)
lstm_df.columns = ['TICKER', 'mse_lstm', 'mape_lstm', 'dir_lstm']

prophet_df = prophet_df.drop('Unnamed: 0', axis=1)
prophet_df.columns = ['TICKER', 'mse_prophet', 'mape_prophet', 'dir_prophet']

# Merge all DataFrames on 'TICKER' to create a comprehensive DataFrame for all methods
merged_df = arima_df.merge(gru_df, on='TICKER').merge(lstm_df, on='TICKER').merge(prophet_df, on='TICKER')

merged_df

,TICKER,mse_arima,mape_arima,dir_arima,mse_gru,mape_gru,dir_gru,mse_lstm,mape_lstm,dir_lstm,mse_prophet,mape_prophet,dir_prophet
0,APD,665.715933,8.299422,0.546296,118.793511,2.680679,0.456221,68.443804,2.332429,0.419355,198.061393,3.976113,0.532407
1,CRH,172.196044,23.003412,0.398148,23.501146,7.288117,0.506912,4.066552,3.185841,0.539171,65.854811,12.769471,0.407407
2,FCX,11.753092,7.081314,0.004630,3.879286,4.332345,0.534562,3.882114,3.984385,0.520737,18.490965,8.533245,0.509259
3,DOW,19.712969,6.540065,0.013889,15.681953,5.917929,0.488479,10.606326,4.507634,0.483871,109.307711,18.532557,0.527778
4,ECL,656.001794,13.159487,0.532407,13.626551,1.769758,0.442396,56.935220,3.788670,0.470046,2276.791119,28.033827,0.462963
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,EXC,7.207220,5.746913,0.023148,2.562569,3.160724,0.488479,2.771071,3.158672,0.465438,8.551827,5.922744,0.495370
104,SO,11.752875,4.084904,0.490741,7.801927,3.488118,0.502304,4.093743,2.294793,0.520737,8.940615,3.497039,0.555556
105,PCG,0.603074,3.941686,0.453704,0.983510,4.460174,0.552995,1.610043,6.574091,0.502304,33.857689,34.328294,0.430556
106,SRE,10.868556,3.899966,0.495370,2.988727,1.967912,0.511521,5.720095,2.620312,0.447005,30.941998,6.596207,0.453704


## Calculate the weights for each models for ticker based on mse and mape

In [ ]:
# Copy relevant columns for weight calculation
weights = merged_df[['mse_arima', 'mape_arima', 'dir_arima', 'mse_gru', 'mape_gru', 'dir_gru', 'mse_lstm', 'mape_lstm', 'dir_lstm', 'mse_prophet', 'mape_prophet', 'dir_prophet']].copy()

# Invert MSE and MAPE values for weighting (since lower values are better)
weights[['mse_arima', 'mape_arima', 'mse_gru', 'mape_gru',  'mse_lstm', 'mape_lstm', 'mse_prophet', 'mape_prophet']]= weights[['mse_arima', 'mape_arima', 'mse_gru', 'mape_gru',  'mse_lstm', 'mape_lstm', 'mse_prophet', 'mape_prophet']].apply(lambda x: 1 / x)

# Apply a scaling factor to directional accuracy to adjust its impact
dir_acc_scale_factor = 0.2

# Scale the directional accuracy by the factor
weights['dir_arima_scaled'] = weights['dir_arima'] * dir_acc_scale_factor
weights['dir_gru_scaled'] = weights['dir_gru'] * dir_acc_scale_factor
weights['dir_lstm_scaled'] = weights['dir_lstm'] * dir_acc_scale_factor
weights['dir_prophet_scaled'] = weights['dir_prophet'] * dir_acc_scale_factor

# Calculate combined weights for each model, then normalize these weights
weights['weight_arima'] = weights[['mse_arima', 'mape_arima', 'dir_arima_scaled']].mean(axis=1)
weights['weight_gru'] = weights[['mse_gru', 'mape_gru', 'dir_gru_scaled']].mean(axis=1)
weights['weight_lstm'] = weights[['mse_lstm', 'mape_lstm', 'dir_lstm_scaled']].mean(axis=1)
weights['weight_prophet'] = weights[['mse_prophet', 'mape_prophet', 'dir_prophet_scaled']].mean(axis=1)

# Normalize the weights
weights_normalized = weights[['weight_arima', 'weight_gru', 'weight_lstm', 'weight_prophet']].div(weights[['weight_arima', 'weight_gru', 'weight_lstm', 'weight_prophet']].sum(axis=1), axis=0)

# Add 'TICKER' column to normalized weights and reorder columns for clarity
weights_normalized['TICKER'] = merged_df['TICKER']
weights_normalized = weights_normalized[['TICKER', 'weight_arima', 'weight_gru', 'weight_lstm', 'weight_prophet']]

# Save the weights to a CSV for use in ensemble modeling
weights_normalized.to_csv("model_weight.csv", index=False)

# Display the final DataFrame of normalized weights
weights_normalized

,TICKER,weight_arima,weight_gru,weight_lstm,weight_prophet
0,APD,0.145058,0.296513,0.330710,0.227720
1,CRH,0.102908,0.224437,0.532970,0.139685
2,FCX,0.132992,0.348543,0.358613,0.159853
3,DOW,0.184555,0.295454,0.369185,0.150805
4,ECL,0.130021,0.513673,0.265359,0.090948
...,...,...,...,...,...
103,EXC,0.139382,0.353219,0.338391,0.169009
104,SO,0.191390,0.230418,0.350639,0.227554
105,PCG,0.457987,0.309098,0.199805,0.033110
106,SRE,0.193464,0.408570,0.279222,0.118744


# Create an ensemble model based on models' weight

## Create a dataframe for ensemble model

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Load the weights data
weights_df = pd.read_csv('model_weight.csv')

# Directory paths
stock_dataset = './data/stock_price_data'
prediction_dir = './prediction'
ensemble_dir = './ensemble'

# Ensure ensemble directory exists
if not os.path.exists(ensemble_dir):
    os.makedirs(ensemble_dir)

# List of models
models = ['prophet', 'arima', 'lstm', 'gru']

def process_ticker(ticker, weights_df):
    # Load actual stock data
    stock_file = f'{stock_dataset}/{ticker}.csv'
    stock_df = pd.read_csv(stock_file)

    # Initialize DataFrame for merged data
    merged_df = stock_df[['Date', 'Close']].tail(217)
    merged_df['Date'] = pd.to_datetime(merged_df['Date'])

    # Load predictions and merge
    for model in models:
        model_file = f'{prediction_dir}/{ticker}_{model}.csv'
        model_df = pd.read_csv(model_file)
        merged_df[model.capitalize()] = model_df['yhat'].head(217).values

    # Calculate ensemble predictions
    ticker_weights = weights_df[weights_df['TICKER'] == ticker].iloc[0]
    ensemble_predictions = sum(ticker_weights[f'weight_{model}'] * merged_df[model.capitalize()] for model in models)

    # Add ensemble predictions to DataFrame
    merged_df['Ensemble'] = ensemble_predictions

    # Save merged data to CSV
    merged_file_path = os.path.join(ensemble_dir, f'{ticker}_merged.csv')
    merged_df.to_csv(merged_file_path, index=False)

# Process each ticker
for ticker in weights_df['TICKER']:
    process_ticker(ticker, weights_df)


## Calculate MAPE for validation set

In [ ]:
# Directory for the merged ensemble files
ensemble_dir = './ensemble'

# Initialize DataFrame to store MAPE for each ticker
mape_df = pd.DataFrame(columns=['TICKER', 'MAPE'])

# Process each file in the ensemble directory
for file in os.listdir(ensemble_dir):
    if file.endswith("_merged.csv"):
        ticker = file.replace('_merged.csv', '')
        file_path = os.path.join(ensemble_dir, file)

        # Read the merged ensemble file
        merged_df = pd.read_csv(file_path)

        # Calculate MAPE using the custom function
        actual_prices = merged_df['Close'].values
        ensemble_predictions = merged_df['Ensemble'].values
        mape = get_mape(actual_prices, ensemble_predictions)

        # Append MAPE to the DataFrame
        new_row = pd.DataFrame({'TICKER': [ticker], 'MAPE': [mape]})
        mape_df = pd.concat([mape_df, new_row], ignore_index=True)

# Save the MAPE DataFrame to CSV
mape_file_path = 'validation_mape.csv'
mape_df.to_csv(mape_file_path, index=False)
